# Práctica 5: Pruebas estadísticas

En esta práctica se retomar las conclusiones de las practicas pasadas, las cuales están basadas en simples observaciones gráficas y de estadística descriptiva, para establecer preguntas que sean contestadas con el uso de las pruebas de hipótesis.

Se inicia delimitando el trabajo en base a las observaciones de los niveles de cada factor:

In [19]:
alphas=[0.0001,1.0]
neuronas_num=[150,210]
layer_num=[1]

In [55]:
import matplotlib as mpl 
import matplotlib.pyplot as plt 
from statsmodels.graphics.gofplots import qqplot
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
from scipy import special
import pandas as pd
pd.set_option('max_rows', 10)
import ssl
py.offline.init_notebook_mode(connected=True)

In [49]:
d = pd.read_csv("../lbfgs_crudo.csv") 

In [50]:
d

,Unnamed: 0,partition,layer_num,neurona_num,activation,alpha,train_accuracy,training_time,test_accuracy
0,1728,10,1,30,logistic,0.0001,0.927833,23.156492,0.8789
1,1729,10,1,30,logistic,1.0000,0.950167,21.880265,0.8949
2,1730,10,1,30,logistic,10000.0000,0.802000,6.253907,0.8065
3,1731,10,1,30,tanh,0.0001,0.926333,22.267032,0.8854
4,1732,10,1,30,tanh,1.0000,0.910000,22.207077,0.8725
...,...,...,...,...,...,...,...,...,...
8625,29563,90,10,210,logistic,1.0000,0.100000,30.609275,0.0974
8626,29564,90,10,210,logistic,10000.0000,0.100000,152.781862,0.0958
8627,29565,90,10,210,tanh,0.0001,0.999741,608.635419,0.9586
8628,29566,90,10,210,tanh,1.0000,0.999667,667.529777,0.9548


Esto implica que ahora se necesita un menor numero de graficar para cubrir los niveles restantes de cada factor como se ve a continuación. Para iniciar los histogramas son una herramienta que permite visualizar como es la distribución de los datos, y de nuevo se hace uso de la librería [**plotly**]( https://plot.ly/python/histograms/) para generar las gráficas:

In [74]:
figs=dict()
neuronas=dict()
for capas in layer_num:
    for i in d.activation.unique():
        for j in alphas:
            for k in neuronas_num:
                neuronas[str(k)+'neuronas,'+str(i)+',a='+str(j)]=list()
                for z in d.test_accuracy[(d.partition==10) & (d.activation==i) & (d.alpha==j) &\
                                             (d.neurona_num==k) & (d.layer_num==capas)]:
                        neuronas[str(k)+'neuronas,'+str(i)+',a='+str(j)].append(z)
            
            traces=dict()
            for k in neuronas_num:
                traces[str(k)]=go.Histogram(
                    x=np.array(neuronas[str(k)+'neuronas,'+str(i)+',a='+str(j)]),
                    name=str(k)+' neuronas',
                    opacity=0.75)

            datos=list()
            for k in traces:
                datos.append(traces[k])
            
            layout = go.Layout(
                title='particion=10%, activacion='+str(i)+', alpha='+str(j),
                barmode='overlay')
            
            figs['10_'+str(i)+'_'+str(j)] = go.Figure(data=datos, layout=layout)
            
for i in figs:
    py.offline.iplot(figs[i], filename='overlaid histogram')

Los histogramas presentan una forma característica de la distribución normal, por ello se procede aplicar una prueba de **Shapiro** para establecer si provienen de una distribución normal o no, en esta ocasión se usa la librería [**SciPy**]( https://docs.scipy.org/doc/scipy/reference/index.html) la cual provee a Python de modelo estadísticos:

In [76]:
from scipy.stats import shapiro
for i in d.activation.unique():
    for j in alphas:
        for k in neuronas_num:
            data=neuronas[str(k)+'neuronas,'+str(i)+',a='+str(j)]
            for lim in [0.05, 0.01]:
                s, p = shapiro(data)
                print('\nactivation={},alpha={},neuronas={},p-value={:.3f}'.format(i,j,k,p))
                if p > lim:
                    print('aceptablemente normal con nivel de significancia', lim)
                else:
                    print('no parece ser normal con nivel de significancia', lim)


activation=logistic,alpha=0.0001,neuronas=150,p-value=0.464
aceptablemente normal con nivel de significancia 0.05

activation=logistic,alpha=0.0001,neuronas=150,p-value=0.464
aceptablemente normal con nivel de significancia 0.01

activation=logistic,alpha=0.0001,neuronas=210,p-value=0.496
aceptablemente normal con nivel de significancia 0.05

activation=logistic,alpha=0.0001,neuronas=210,p-value=0.496
aceptablemente normal con nivel de significancia 0.01

activation=logistic,alpha=1.0,neuronas=150,p-value=0.142
aceptablemente normal con nivel de significancia 0.05

activation=logistic,alpha=1.0,neuronas=150,p-value=0.142
aceptablemente normal con nivel de significancia 0.01

activation=logistic,alpha=1.0,neuronas=210,p-value=0.819
aceptablemente normal con nivel de significancia 0.05

activation=logistic,alpha=1.0,neuronas=210,p-value=0.819
aceptablemente normal con nivel de significancia 0.01

activation=tanh,alpha=0.0001,neuronas=150,p-value=0.322
aceptablemente normal con nivel de 

En base a los resultados se puede concluir que los datos provienen de una distribución normal y por lo tanto es posible usar herramientas que nos permitan saber si los niveles de un factor afectan al rendimiento. Para este punto se usa una prueba de hipótesis de T-test.

In [85]:
from scipy.stats import ttest_ind
for i in d.activation.unique():
    for j in alphas:
        for lim in [0.05,0.01]:
            t, p = ttest_ind(neuronas['150neuronas,'+str(i)+',a='+str(j)],neuronas['210neuronas,'+str(i)+',a='+str(j)])
            if p>lim:
                print('la prueba de hipotesis indica que SI existe una diferencia significativa p-value={} entre usar \
                150 y 210 neuronas con los parametros activacion={} y alpha={}\n'.format(p,i,j))
            else:
                print('la prueba de hipotesis indica que NO existe una diferencia significativa p-value={} entre usar \
                150 y 210 neuronas con los parametros activacion={} y alpha={}\n'.format(p,i,j))

la prueba de hipotesis indica que NO existe una diferencia significativa p-value=0.03459225629714305 entre usar                 150 y 210 neuronas con los parametros activacion=logistic y alpha=0.0001

la prueba de hipotesis indica que SI existe una diferencia significativa p-value=0.03459225629714305 entre usar                 150 y 210 neuronas con los parametros activacion=logistic y alpha=0.0001

la prueba de hipotesis indica que NO existe una diferencia significativa p-value=0.04493984081525475 entre usar                 150 y 210 neuronas con los parametros activacion=logistic y alpha=1.0

la prueba de hipotesis indica que SI existe una diferencia significativa p-value=0.04493984081525475 entre usar                 150 y 210 neuronas con los parametros activacion=logistic y alpha=1.0

la prueba de hipotesis indica que NO existe una diferencia significativa p-value=0.00016326179797853565 entre usar                 150 y 210 neuronas con los parametros activacion=tanh y alpha=0.000

Podemos concluir que para el caso de entrenar con una partición del 10% de todo el conjunto de datos disponibles, los mejores resultados se obtienen al utilizar 150 0 210 neuronas por cada capa oculta, los valores de alpha 0.0001 y 1.0, y la función de activación `tanh` y `logistic`. Esto porque la prueba de hipótesis apoya la idea que la combinación de estos niveles no produce un cambio sobre el rendimiento.